# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [91]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [92]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'


for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [93]:
full_data_rows_list = [] 
    

for f in file_path_list:


    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
       
        for line in csvreader:
            
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


#### Check the number of rows in the file

In [94]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin Cassandra Setup

#### Creating a Cluster

In [95]:
from cassandra.cluster import Cluster
cluster = Cluster()


session = cluster.connect()

#### Create Keyspace

In [96]:
try:
    session.execute("""DROP KEYSPACE IF EXISTS music """
)

except Exception as e:
    print(e)


try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [97]:
try:
    session.set_keyspace('music')
except Exception as e:
    print(e)

#### Creating and Populating the "songs_by_user_session" table
##### Modeling the table in respect to query (Give me the artist, song title and song's length in the music app history that was heard during sessionID = 338, and itemInSession = 4)
##### For "songs_by_user_session", the sessionid and iteminsession was used as Primary Key because the query will filter these two columns.

In [99]:
query = "DROP TABLE songs_by_user_session"
try:
    session.execute(query)
except Exception as e:
    print(e)


query = "CREATE TABLE IF NOT EXISTS songs_by_user_session ( \
            sessionid int, \
            iteminsession int, \
            artist text, \
            song text, \
            length float, \
            PRIMARY KEY(sessionid, iteminsession))"

try:
    session.execute(query)
except Exception as e:
    print(e)   

#### Extract data from csv file and use to populate the "songs_by_user_session" table

In [100]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_user_session (sessionid, iteminsession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Performing a query on the "songs_by_user_session" table 
##### (Give me the artist, song title and song's length in the music app history that was heard duing sessionID = 338 and itemInSession = 4)

In [101]:
query = "SELECT artist, song, length \
            FROM songs_by_user_session \
            WHERE sessionid=338 AND iteminsession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)    

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Creating "user_and_song_by_id" table
##### (Query the name of artist, song (sorted by itemInSession) and user (firstname and lastname) for userId = 10, sessionId = 182))
##### For "user_and_song_by_id", the userid and sessionid was used as Primary Key because the query will filter these two columns and the iteminsession was used as clustering columns along with artist, firstname, song, lastname as data.

In [103]:
query = "DROP TABLE user_and_song_by_id"
try:
    session.execute(query)
except Exception as e:
    print(e)


query = "CREATE TABLE IF NOT EXISTS user_and_song_by_id ( \
        userid int, \
        sessionid int, \
        iteminsession int, \
        artist text, \
        song text, \
        firstname text, \
        lastname text, \
        PRIMARY KEY((userid, sessionid), iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_and_song_by_id (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))                         

#### Perform a query on the "user_and_song_by_id" table 


In [104]:
query = "SELECT userid, sessionid, artist, song, iteminsession, firstname, lastname \
         FROM user_and_song_by_id  \
         WHERE userid=10 AND sessionid=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userid, row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

10 0 Down To The Bone Keep On Keepin' On Sylvie Cruz
10 1 Three Drives Greece 2000 Sylvie Cruz
10 2 Sebastien Tellier Kilometer Sylvie Cruz
10 3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Creating "usernames" table
##### (Query give me every user name (first and last) in the music app history who listened to the song 'All Hands Aginst His Own'))
##### For "usernames, the song was used as partition key with the userid used as clustering column (sorting column). firstname and lastname as data.

In [115]:
query = "DROP TABLE usernames"
try:
    session.execute(query)
except Exception as e:
    print(e)


query = "CREATE TABLE IF NOT EXISTS usernames ( \
                  song text, \
                  userid int, \
                  firstname text, \
                  lastname text, \
                  PRIMARY KEY(song, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO usernames (song, userid, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))                   

#### Perform a query on the "usernames" table 
##### (Select every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own')

In [116]:
query = "select song, userid, firstname, lastname \
         FROM usernames \
         WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song, row.userid, row.firstname, row.lastname)

All Hands Against His Own 29 Jacqueline Lynch
All Hands Against His Own 80 Tegan Levine
All Hands Against His Own 95 Sara Johnson


### Drop the tables

In [89]:
query = "DROP TABLE songs_by_user_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE user_and_song_by_id"
try:
    session.execute(query)
except Exception as e:
    print(e)
 
query = "DROP TABLE usernames"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [90]:
session.shutdown()
cluster.shutdown()